## IMU Validation Overground Running

IMeasureU Blue Thunder Specs:
- Sampling freq for lowg = 500hz
- 5 minutes of data = 337,500 rows (1125hz) or 478,000 (1600hz)

Overview:
- The purpose of this script is to derive data from the IMU which correspondes to a footstrike on the force plate
- The is done using an "offset time" from when the subject stomps on the ground

In [ ]:
# Import custom functions ---
import functions.file_import_gui as gui
import functions.data_prep as prep
import functions.custom_plots as plots

# For saving files
import os

# For dataframes ---
import pandas as pd

# For plotting ---
import plotly.graph_objs as go
from plotly.subplots import make_subplots

In [ ]:
# Subject to process
sub_id = 'imu_val_001'
run_type = 'run'

# set directory
initialdir = f"data/validity_og/individual_trials/{sub_id}/{run_type}"
# bring in csv files with data
dfs_trials, keys_list = gui.read_csv_files_gui(initialdir)

# subject's offset times
filepath = f"data/validity_og/offset_times/{run_type}/{sub_id}.csv"
# Read the csv file
offset_times_df = pd.read_csv(filepath)

# keep only the "good" trials
dfs_good_trials = prep.filter_out_dfs(dfs_trials, offset_times_df)

Prep

In [ ]:
# calculate and add resultant column
prep.add_resultant_column(dfs_good_trials, column_x = 'accel_x (m/s2)', column_y = 'accel_y (m/s2)', column_z = 'accel_z (m/s2)', name_of_res_column = 'res_m/s/s')
# convert accel columns to gs
prep.accel_to_gs_columns(dfs_good_trials, column_x = 'accel_x (m/s2)', column_y = 'accel_y (m/s2)', column_z = 'accel_z (m/s2)', name_of_res_column = 'res_m/s/s')
# finds window for force plate foot strike using foot stomp (initial peak) and offset times
offset_windows_df = prep.peak_and_window_data(dfs_good_trials, offset_times_df, search_window_margin=50)

In [ ]:
# Create dictionary that stores each IMU together with the trial number

from collections import defaultdict

# Initialize dictionaries to hold the separated trials
dfs_trials_separate = defaultdict(dict)

# Iterate through the items in your original dictionary
for key, df in dfs_good_trials.items():
    # Extract trial number from the key
    # Assuming the format is consistent and the trial number is always at the end
    trial_number = key.split('_')[-1]

    # Add the dataframe to the corresponding trial dictionary
    dfs_trials_separate[trial_number][key] = df

Visually Inspect & Find Peaks

In [ ]:
# Plot each IMU by trial number
plots.plot_trial_data(dfs_trials_separate, 'trial6', 'res_g', offset_windows_df=offset_windows_df)